In [11]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [12]:
from langchain_community.vectorstores import Chroma

CHROMA_DB_PATH = "./chroma_db"
COLLECTION_NAME = "pdf_docs"

def get_vectorstore():
    return Chroma(
        collection_name=COLLECTION_NAME,
        embedding_function=embeddings,
        persist_directory=CHROMA_DB_PATH
    )

In [13]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.1")

In [14]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def build_rag_chain():
    vectorstore = get_vectorstore()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    prompt = PromptTemplate.from_template("""
Usa el siguiente contexto para responder la pregunta.
Si no hay suficiente información, responde claramente que no está en el documento.

Contexto:
{context}

Pregunta:
{question}

Respuesta:
""")

    format_docs = lambda docs: "\n\n".join(d.page_content for d in docs)

    chain = (
        RunnableParallel(
            context=retriever | format_docs,
            question=RunnablePassthrough()
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

In [15]:
def rag_chat(question):
    chain = build_rag_chain()
    return chain.invoke(question)

In [17]:
respuesta = rag_chat("¿Cuál es la tesis de este artículo?")
print(respuesta)

No está claramente expresada en el texto proporcionado, pero se puede inferir que el autor (Carlos Periñán Pascual) defiende el procesamiento del lenguaje natural basado en modelos semánticos y critica a los sistemas de la década de 1960 como ELIZA por no fundamentarse en ningún modelo semántico. El artículo parece defender una postura más estructurada y formal en relación con el procesamiento del lenguaje natural, distanciándose de los sistemas que simplemente reconocían patrones superficiales y recreaban respuestas sin análisis semántico profundo.
